Model Training

1.1 Import Data and Required Packages

Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [59]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [60]:
df = pd.read_csv('raw.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [61]:
X = df.drop(['math_score'], axis=1)
X.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [62]:
y= df['math_score']
y.head()

0    72
1    69
2    90
3    47
4    76
Name: math_score, dtype: int64

In [63]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

In [64]:
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
encoding_transformer = OneHotEncoder()

preprocessor =ColumnTransformer(
    [
        ('OneHotEncoder', encoding_transformer, cat_features),
        ('StandardScaler', numeric_transformer, num_features),
    ]
)

In [65]:
X = preprocessor.fit_transform(X)

In [66]:
X.shape

(1001, 19)

In [67]:
# separate dataset into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((800, 19), (201, 19))

Create an Evaluate Function to give all metrics after model Training

In [68]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square


In [69]:
models= {
    "Linear Regression" : LinearRegression(),
    "Lasso" : Lasso(),
    "Ridge" : Ridge(),
    "K-Neighbor" : KNeighborsRegressor(),
    "Decision Tree Regressor" : DecisionTreeRegressor(),
    "Random Forest Regressor" : RandomForestRegressor(),
    "XGB Regressor" : XGBRegressor(),
    "CatBoost Regressor" : CatBoostRegressor(allow_writing_files=False,silent=True),
    "AdaBoost Regressor" : AdaBoostRegressor()
}

In [70]:
model_list = []
r2_list = []

for i in range(len(models)):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    #Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_mae , train_rmse, train_r2 = evaluate_model(y_train, y_train_pred)
    test_mae , test_rmse, test_r2 = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance for training set:')
    print('RMSE: {}'.format(train_rmse))
    print('MAE: {}'.format(train_mae))
    print('R2 Score: {}'.format(train_r2))

    print('----------------------------------')
    
    print('Model performance for Test set')
    print("RMSE: {}".format(test_rmse))
    print("MAE: {}".format(test_mae))
    print("R2 Score: {}".format(test_r2))
    r2_list.append(test_r2)
    
    print('='*35)
    print('\n')



Linear Regression
Model performance for training set:
RMSE: 5.306300758662891
MAE: 4.269611053466797
R2 Score: 0.8729802664056152
----------------------------------
Model performance for Test set
RMSE: 5.433877458679674
MAE: 4.158565995705068
R2 Score: 0.8858027346969402


Lasso
Model performance for training set:
RMSE: 6.56135155828919
MAE: 5.206173645021661
R2 Score: 0.8057888814652769
----------------------------------
Model performance for Test set
RMSE: 6.66358891955486
MAE: 5.145619867969862
R2 Score: 0.8282675292988068


Ridge
Model performance for training set:
RMSE: 5.303150777451433
MAE: 4.268617298053178
R2 Score: 0.8731310271837334
----------------------------------
Model performance for Test set
RMSE: 5.4303548323037685
MAE: 4.157447041792989
R2 Score: 0.8859507483061705


K-Neighbor
Model performance for training set:
RMSE: 5.866979631803744
MAE: 4.6617500000000005
R2 Score: 0.8447196023255863
----------------------------------
Model performance for Test set
RMSE: 7.03684

Random Forest Regressor
Model performance for training set:
RMSE: 2.2781686752261505
MAE: 1.8136159226190478
R2 Score: 0.9765868835236218
----------------------------------
Model performance for Test set
RMSE: 6.475246594248706
MAE: 4.973117744610283
R2 Score: 0.837838165917443


XGB Regressor
Model performance for training set:
RMSE: 0.9627133342452312
MAE: 0.6380063438415527
R2 Score: 0.995818987673936
----------------------------------
Model performance for Test set
RMSE: 7.0481734598010615
MAE: 5.398202174931616
R2 Score: 0.8078726507197334


CatBoost Regressor
Model performance for training set:
RMSE: 3.066440875003814
MAE: 2.4085236439395414
R2 Score: 0.957581367425283
----------------------------------
Model performance for Test set
RMSE: 6.154418257737495
MAE: 4.658982145461262
R2 Score: 0.8535093051401479


AdaBoost Regressor
Model performance for training set:
RMSE: 5.815055770080755
MAE: 4.769325530850952
R2 Score: 0.847455960706514
----------------------------------
Model p

In [71]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)


,Model Name,R2_Score
2,Ridge,0.885951
0,Linear Regression,0.885803
7,CatBoost Regressor,0.853509
5,Random Forest Regressor,0.837838
1,Lasso,0.828268
8,AdaBoost Regressor,0.822500
3,K-Neighbor,0.808490
6,XGB Regressor,0.807873
4,Decision Tree Regressor,0.726385
